In [ ]:
# Create/load profile/computer/codes
import utilities_aiida as ua

from common import VISCOSITY_CODES


print("Ensure required computer/codes are set up...")

profile = ua.setup_profile()
computer = ua.setup_computer()
codes = ua.setup_codes(code_infos=VISCOSITY_CODES)

print("Using profile:")
print(f" - {profile}")
print("Using computer:")
print(f" - {computer}")
print("Using codes: ")
for code in codes:
    print(f" - {code}")

# create_codes_button.on_click(on_create_codes_button_clicked)
# display(create_codes_button, output_area)

In [ ]:
# Widgets for launching a Monomer Viscosity Workchain
import ipywidgets as wdg

from aitw_aiida_viscosity.cli.workflows.viscosity import launch_workflow

from common import VISCOSITY_WORKCHAIN_DEFAULTS, VISCOSITY_CODE_MAP
import utilities
import utilities_aiida as ua


title = wdg.HTML(value="<h2>AITW - AiiDA Monomer Viscosity Workchain</h2>")
display(title)
try:
    with ua.with_aiida_profile():
        widgets_launch = utilities.get_widgets_from_click_function(
            launch_workflow,
            override_defaults=VISCOSITY_WORKCHAIN_DEFAULTS,
            code_map=VISCOSITY_CODE_MAP,
        )
except Exception as e:
    print("Error:", str(e))
else:
    widget_box = wdg.VBox(list(widgets_launch.values()))
    display(widget_box)

    generate_button = wdg.Button(description="Launch Workchain", button_style="success")
    output_area = wdg.Output()

    def on_generate_button_clicked(b):
        with output_area:
            output_area.clear_output()
            args = utilities.get_click_args_from_widgets(launch_workflow, widgets_launch)
            launch_workflow(args, standalone_mode=False)

    generate_button.on_click(on_generate_button_clicked)
    display(generate_button)
    display(output_area)

In [ ]:
# Widget for plotting viscosity data produced by the Monomer Viscosity Workchain
import pathlib
import ipywidgets as wdg
from IPython.display import Image

from aiida import orm

from aitw_aiida_viscosity.cli.data.viscosity_array import plot_viscosity
from utilities_aiida import with_aiida_profile

choices = []
node_wc_data = {}
with with_aiida_profile():
    qb = orm.QueryBuilder()
    qb.append(orm.ArrayData, filters={'attributes': {'has_key': 'array|deformation_velocities'}})
    for node, in qb.all():
        pk = node.pk
        wc, _, _ = node.base.links.get_incoming(link_label_filter='viscosity_data').first()
        ptr = node_wc_data[pk] = {}

        for inp_field in wc.inputs:
            inp_node = getattr(wc.inputs, inp_field)
            try:
                value = inp_node.value
            except AttributeError:
                value = str(inp_node)
            ptr[inp_field] = value
        smile_str = wc.inputs.smiles_string.value
        def_vel = wc.inputs.deform_velocities.value
        nmols = wc.inputs.nmols.value
        num_steps = wc.inputs.num_steps.value
        time_step = wc.inputs.time_step.value
        time_ps = num_steps * time_step

        choices.append((
            f"{smile_str} | nmols: {nmols} | Def Vel: {def_vel} | time (ps): {time_ps:.3f}",
            pk
        ))
        
widgets_show = []
widgets_show.append(
    wdg.Dropdown(
        options=choices,
        description='Select Viscosity Data:',
        disabled=False,
        style={'description_width': '150px'},
    )
)

data_box = wdg.VBox(widgets_show)
display(data_box)
plot_button = wdg.Button(description="Plot Viscosity Data", button_style="info")

output_area = wdg.Output()

def on_plot_button_clicked(b):
    result_dir = pathlib.Path("viscosity_plots")
    result_dir.mkdir(exist_ok=True)
    with output_area:
        output_area.clear_output()
        selected_data = widgets_show[0].value
        pk = selected_data
        filename = (result_dir / f"viscosity_data_{pk}.png").as_posix()

        print(f"Plotting Viscosity from workchain with inputs:")
        for inp_field, value in node_wc_data[pk].items():
            print(f" - {inp_field:>30s}: {value}")

        plot_viscosity([str(pk), '-o', filename], standalone_mode=False)
        display(Image(filename=filename))
    

plot_button.on_click(on_plot_button_clicked)
display(plot_button)
display(output_area)